<a href="https://colab.research.google.com/github/NiyazovIlia/PyTorch-1/blob/lesson-4/HW_PyTorch_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)


class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

Files already downloaded and verified


In [3]:
trans_actions = transforms.Compose([transforms.Resize(40),
                                    transforms.RandomCrop(32, padding=3), 
                                    transforms.ToTensor()])


def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.05, random_state=13)
    return X_train, X_test

train_dataset, valid_dataset = train_valid_split(dataset)

In [4]:
train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=250,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=250,
                          shuffle=False,
                          num_workers=1)

In [6]:
classes = ['apple','aquarium_fish','baby','bear','beaver','bed','bee','beetle','bicycle','bottle','bowl','boy','bridge','bus','butterfly',
           'camel','can','castle','caterpillar','cattle','chair','chimpanzee','clock','cloud','cockroach','couch','crab','crocodile','cup',
           'dinosaur','dolphin','elephant','flatfish','forest','fox','girl','hamster','house','kangaroo','keyboard','lamp','lawn_mower',
           'leopard','lion','lizard','lobster','man','maple_tree','motorcycle','mountain','mouse','mushroom','oak_tree','orange',
           'orchid','otter','palm_tree','pear','pickup_truck','pine_tree','plain','plate','poppy','porcupine','possum','rabbit',
           'raccoon','ray','road','rocket','rose','sea','seal','shark','shrew','skunk','skyscraper','snail','snake','spider','squirrel',
           'streetcar','sunflower','sweet_pepper','table','tank','telephone','television','tiger','tractor','train','trout','tulip',
           'turtle','wardrobe','whale','willow_tree','wolf','woman','worm']
len(classes)

100

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.dp_one = nn.Dropout(0.2)
        self.dp_two = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, kernel_size=3,padding=1)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 90, kernel_size=3,padding=1)
        self.bn_three = torch.nn.BatchNorm2d(90)
        self.conv_three = torch.nn.Conv2d(90, 120, kernel_size=3,padding=1)

        self.bn_four = torch.nn.BatchNorm2d(120)
        self.conv_four = torch.nn.Conv2d(120, 160, kernel_size=3, padding=1)

        self.bn_five = torch.nn.BatchNorm2d(160)
        self.fc1 = torch.nn.Linear(640, 200)
        self.fc2 = torch.nn.Sigmoid()
        self.out = torch.nn.Linear(200, 100)
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)

        x = self.bn_four(x)
        x = self.conv_four(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_five(x)
        x = x.view(x.size(0), -1)
        x = self.dp_one(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_two(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)
       
net = Net().to(device)
print(net)

Net(
  (dp_one): Dropout(p=0.2, inplace=False)
  (dp_two): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 90, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_three): BatchNorm2d(90, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(90, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_four): Conv2d(120, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_five): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=640, out_features=200, bias=True)
  (fc2): Sigmoid()
  (out): Linear(in_features=200, out_fea

In [13]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [14]:
num_epochs = 10
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 50 == 0:    # печатаем каждые 50 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/190]. Loss: 0.019. Acc: 0.004. Test acc: 0.011
Epoch [1/10]. Step [51/190]. Loss: 0.018. Acc: 0.028. Test acc: 0.049
Epoch [1/10]. Step [101/190]. Loss: 0.017. Acc: 0.053. Test acc: 0.064
Epoch [1/10]. Step [151/190]. Loss: 0.016. Acc: 0.065. Test acc: 0.080
Epoch [2/10]. Step [1/190]. Loss: 0.016. Acc: 0.080. Test acc: 0.071
Epoch [2/10]. Step [51/190]. Loss: 0.016. Acc: 0.082. Test acc: 0.080
Epoch [2/10]. Step [101/190]. Loss: 0.016. Acc: 0.089. Test acc: 0.109
Epoch [2/10]. Step [151/190]. Loss: 0.015. Acc: 0.100. Test acc: 0.106
Epoch [3/10]. Step [1/190]. Loss: 0.015. Acc: 0.116. Test acc: 0.112
Epoch [3/10]. Step [51/190]. Loss: 0.015. Acc: 0.119. Test acc: 0.100
Epoch [3/10]. Step [101/190]. Loss: 0.015. Acc: 0.118. Test acc: 0.106
Epoch [3/10]. Step [151/190]. Loss: 0.015. Acc: 0.124. Test acc: 0.132
Epoch [4/10]. Step [1/190]. Loss: 0.014. Acc: 0.160. Test acc: 0.142
Epoch [4/10]. Step [51/190]. Loss: 0.014. Acc: 0.143. Test acc: 0.156
Epoch [4/10]. Step